# Analysis 2a

Compare the outcomes of the elections for random and honest voters and candidates, with agents distributed uniformly in the opinion space.

### Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cd ..

In [ ]:
from population import Population
from simulation import Simulation
from plotting import *
from geometry import *
from election import Election
import copy

from agents import Voter, Candidate, System, Strategy, Approach

### Functions

Some example functions to show how to extract data from the simulation.



In [ ]:
def plot_stats(ax: plt.Axes, res: list, n_sims: int, n_rounds: int, pop_name: str):
    vse_comp, vse_util, vse_vdist_comp, norm_entropy = np.zeros((n_sims, n_rounds)), np.zeros((n_sims, n_rounds)), np.zeros((n_sims, n_rounds)), np.zeros((n_sims, n_rounds))
    for j, results in enumerate(res):
        vse_comp[j, :] = np.reshape([r.get('vse_comp')  for r in results], (1, n_rounds))
        vse_util[j, :] = np.reshape([r.get('vse_util') for r in results], (1, n_rounds))
        vse_vdist_comp[j, :] = np.reshape([r.get('vse_vdist_comp') for r in results], (1, n_rounds))
        norm_entropy = np.reshape([r.get('norm_entropy') for r in results], (1, n_rounds))
    # average over simulations 
    vse_comp_mean = np.mean(vse_comp, axis=0)
    vse_util_mean = np.mean(vse_util, axis=0)
    vse_vdist_comp_mean = np.mean(vse_vdist_comp, axis=0)
    norm_entropy_mean = np.mean(norm_entropy, axis=0)
    # get std
    vse_comp_std = np.std(vse_comp, axis=0)
    vse_util_std = np.std(vse_util, axis=0)
    vse_vdist_comp_std = np.std(vse_vdist_comp, axis=0)
    norm_entropy_std = np.std(norm_entropy, axis=0)
    # plot results

    ax.plot(vse_comp_mean, label='VSE Comp', color='blue')
    ax.fill_between(range(n_rounds), vse_comp_mean - vse_comp_std, vse_comp_mean + vse_comp_std, color='blue', alpha=0.2)
    ax.plot(vse_util_mean, label='VSE Util', color='orange')
    ax.fill_between(range(n_rounds), vse_util_mean - vse_util_std, vse_util_mean + vse_util_std, color='orange', alpha=0.2)
    ax.plot(vse_vdist_comp_mean, label='VSE VDist Comp', color='green')
    ax.fill_between(range(n_rounds), vse_vdist_comp_mean - vse_vdist_comp_std, vse_vdist_comp_mean + vse_vdist_comp_std, color='green', alpha=0.2)
    ax.plot(norm_entropy_mean, label='Norm Entropy', color='red')
    ax.fill_between(range(n_rounds), norm_entropy_mean - norm_entropy_std, norm_entropy_mean + norm_entropy_std, color='red', alpha=0.2)
    ax.set_title(f'Simulation Results for {pop_name}')
    ax.set_xlabel('Round')
    ax.set_ylabel('Value')
    ax.legend(loc='center right', fontsize='xx-small')
    ax.grid()

def plot_dynamics(ax: plt.Axes, sim_res: list, pop_name: str):
    rounds = len(sim_res)
    results = [result['votes_per'] for result in sim_res]
    cand_results = np.reshape(results[:], (rounds, len(results[0])))

    prev = 0
    for i in range(len(results[0])):
        ax.fill_between(range(rounds), prev, prev+cand_results[:, i], label=f'C {i}', color=COLOURS[i % len(COLOURS)], alpha=0.5)
        prev += cand_results[:, i]

    ax.set_xlabel('Round')
    ax.set_ylabel('Votes')
    ax.set_title(f'Vote Composition for {pop_name}')
    ax.legend(loc='center right', fontsize='xx-small')
    ax.grid()

## Setup

```python

#### Hyperparams

In [ ]:
n_sims = 9
n_rounds = 10

#### Model Parameters

In [ ]:
n_voters = 1000
n_cands = 10

params = {
    'voter_strategy': 'uniform',
    'candidate_approach': 'uniform',
    'n_rounds': n_rounds,
}

#### Set up population of Uniform Random Voters & Random Candidates

In [ ]:
votersH = [Voter(coords=[np.random.normal(0, 1, size=2)], id=i, strat=Strategy.HONEST, params=params) for i in range(n_voters)]
candsO = [Candidate(coords=[float(np.random.uniform(-1, 1)) for _ in range(2)], id=i, approach=Approach.OFFENSIVE) for i in range(n_cands)]
candsD = [Candidate(coords=[float(np.random.uniform(-1, 1)) for _ in range(2)], id=i, approach=Approach.DEFENSIVE) for i in range(n_cands)]
candsM = [Candidate(coords=[float(np.random.uniform(-1, 1)) for _ in range(2)], id=i, approach=(Approach.OFFENSIVE if i%2 == 0 else Approach.DEFENSIVE)) for i in range(n_cands)]

##### Set up populations

In [ ]:
pop1 = Population(voters=votersH, candidates=candsO, params=params)
pop2 = Population(voters=votersH, candidates=candsD, params=params)
pop3 = Population(voters=votersH, candidates=candsM, params=params)

pop_names = ['OFFENSIVE', 'DEFENSIVE', 'MIXED']



# Run Simulations

## FPTP

In [ ]:

res1_FPTP = []
res2_FPTP = []
res3_FPTP = []

for i in range(n_sims):
    pop1_int, pop2_int, pop3_int = copy.copy(pop1), copy.copy(pop2), copy.copy(pop3)
    sim1 = Simulation(population = pop1_int, election=Election(population = pop1_int, params={'system': System.FPTP}))
    sim2 = Simulation(population = pop2_int, election=Election(population = pop2_int, params={'system': System.FPTP}))
    sim3 = Simulation(population = pop3_int, election=Election(population = pop3_int, params={'system': System.FPTP}))
    
    output1 = sim1.run_election_cycles(save_results=False, plot_results=False, make_gif=(i==0), verbose=False)
    output2 = sim2.run_election_cycles(save_results=False, plot_results=False, make_gif=(i==0), verbose=False)
    output3 = sim3.run_election_cycles(save_results=False, plot_results=False, make_gif=(i==0), verbose=False)
    print(f"Simulation {i+1} completed.")
    res1_FPTP.append(output1.get('results'))
    res2_FPTP.append(output2.get('results'))
    res3_FPTP.append(output3.get('results'))

results_FPTP = [res1_FPTP, res2_FPTP, res3_FPTP]


#### Plotting

In [ ]:
# extract stats and plot results
fig1, axs1 = plt.subplots(1, 3, figsize=(20, 5))
fig2, axs2 = plt.subplots(1, 3, figsize=(20, 5))
for i, res in enumerate(results_FPTP):
    plot_stats(axs1[i], res, n_sims, n_rounds, pop_names[i])
    plot_dynamics(axs2[i], res[0], pop_names[i])
fig1.suptitle(f'First-Past-The-Post n={n_voters}, c={n_cands}, r={n_rounds}')
fig2.suptitle(f'First-Past-The-Post n={n_voters}, c={n_cands}, r={n_rounds}')
fig1.savefig('./plots/anal2a/stats_FPTP.png')
fig2.savefig('./plots/anal2a/dynamics_FPTP.png')


### Permanent Plot

![FPTP](../plots/anal2a/stats_fptp.png)

![FPTP dynamics](../plots/anal2a/dynamics_FPTP.png)

#### Clear sim objects

In [ ]:
del pop1_int, pop2_int, pop3_int
del sim1, sim2, sim3
del output1, output2, output3

## Instant Runoff

In [ ]:

res1_IR = []
res2_IR = []
res3_IR = []

for i in range(n_sims):
    pop1_int, pop2_int, pop3_int = copy.copy(pop1), copy.copy(pop2), copy.copy(pop3)
    sim1 = Simulation(population = pop1_int, election=Election(population = pop1_int, params={'system': System.INSTANT_RUNOFF}))
    sim2 = Simulation(population = pop2_int, election=Election(population = pop2_int, params={'system': System.INSTANT_RUNOFF}))
    sim3 = Simulation(population = pop3_int, election=Election(population = pop3_int, params={'system': System.INSTANT_RUNOFF}))

    output1 = sim1.run_election_cycles(save_results=False, plot_results=False, make_gif=(i==0), verbose=False)
    output2 = sim2.run_election_cycles(save_results=False, plot_results=False, make_gif=(i==0), verbose=False)
    output3 = sim3.run_election_cycles(save_results=False, plot_results=False, make_gif=(i==0), verbose=False)

    print(f"Simulation {i+1} completed.")
    res1_IR.append(output1.get('results'))
    res2_IR.append(output2.get('results'))
    res3_IR.append(output3.get('results'))


results_IR = [res1_IR, res2_IR, res3_IR]

#### Plotting

In [ ]:
# extract stats and plot results
fig1, axs1 = plt.subplots(1, 3, figsize=(20, 5))
fig2, axs2 = plt.subplots(1, 3, figsize=(20, 5))
for i, res in enumerate(results_IR):
    plot_stats(axs1[i], res, n_sims, n_rounds, pop_names[i])
    plot_dynamics(axs2[i], res[0], pop_names[i])

fig1.suptitle(f'Instant Runoff n={n_voters}, c={n_cands}, r={n_rounds}')
fig2.suptitle(f'Instant Runoff n={n_voters}, c={n_cands}, r={n_rounds}')
fig1.savefig(f'./plots/anal2a/stats_IR.png')
fig2.savefig(f'./plots/anal2a/dynamics_IR.png')



### Permanent Plots

![Instant Runoff](../plots/anal2a/stats_IR.png)


![Instant Runoff dynamics](../plots/anal2a/dynamics_IR.png)

##### Clear memory of sim objects

In [ ]:
del sim1, sim2, sim3
del output1, output2, output3
del pop1_int, pop2_int, pop3_int

## Approval

In [ ]:
res1_A = []
res2_A = []
res3_A = []

for i in range(n_sims):
    pop1_int, pop2_int, pop3_int = copy.copy(pop1), copy.copy(pop2), copy.copy(pop3)
    sim1 = Simulation(population = pop1_int, election=Election(population = pop1_int, params={'system': System.APPROVAL}))
    sim2 = Simulation(population = pop2_int, election=Election(population = pop2_int, params={'system': System.APPROVAL}))
    sim3 = Simulation(population = pop3_int, election=Election(population = pop3_int, params={'system': System.APPROVAL}))

    
    output1 = sim1.run_election_cycles(save_results=False, plot_results=False, make_gif=(i==0), verbose=False)
    output2 = sim2.run_election_cycles(save_results=False, plot_results=False, make_gif=(i==0), verbose=False)
    output3 = sim3.run_election_cycles(save_results=False, plot_results=False, make_gif=(i==0), verbose=False)

    print(f"Simulation {i+1} completed.")
    res1_A.append(output1.get('results'))
    res2_A.append(output2.get('results'))
    res3_A.append(output3.get('results'))


results_AP = [res1_A, res2_A, res3_A]

#### Plotting

In [ ]:
# extract stats and plot results
fig1, axs1 = plt.subplots(1, 3, figsize=(20, 5))
fig2, axs2 = plt.subplots(1, 3, figsize=(20, 5))
for i, res in enumerate(results_AP):
    plot_stats(axs1[i], res, n_sims, n_rounds, pop_names[i])
    plot_dynamics(axs2[i], res[0], pop_names[i])

fig1.suptitle(f'Approval Voting n={n_voters}, c={n_cands}, r={n_rounds}')
fig2.suptitle(f'Approval Voting n={n_voters}, c={n_cands}, r={n_rounds}')
fig1.savefig(f'./plots/anal2a/stats_AP.png')
fig2.savefig(f'./plots/anal2a/dynamics_AP.png')


### Permanent Plots

![Approval Voting Stats](../plots/anal2a/stats_AP.png)

![Approval Voting](../plots/anal2a/dynamics_AP.png)